In [ ]:
!pip install azure-kusto-data azure-kusto-ingest

In [4]:
from datetime import datetime, timedelta
import os
import pyodbc

SQL_USER="sqlRead"
SQL_PWD="ChangeYourAdminPassword1"
SQL_SERVER="adxdemo.database.windows.net"

# AAD_TENANT_ID = spark.conf.get("trident.tenant.id")
KUSTO_CLUSTER =  "<kusto cluster>"
KUSTO_DATABASE = "ECommerceStore"

today = datetime.now()
print(today.strftime('%Y-%m-%d %H:%M:%S'))

2024-03-31 19:06:29


In [5]:
# get last day the data was copied
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd

kusto_conn = KustoConnectionStringBuilder.with_aad_device_authentication(KUSTO_CLUSTER)
kusto_conn.authority_id = AAD_TENANT_ID
kustoClient = KustoClient(kusto_conn)

StatementMeta(, 614ef43d-92fe-44b7-91e3-5d1c67402521, 9, Finished, Cancelled)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F69DN3QAP to authenticate.


In [40]:
query = f"cdcCheckpoint |where TableName == '{table_name}' | summarize max(LastUpdate) "
results = kusto_client.execute(KUSTO_DATABASE, query)
df = dataframe_from_result_table(results.primary_results[0])
last_copy_date = df['max_LastUpdate'][0]
df

StatementMeta(, 614ef43d-92fe-44b7-91e3-5d1c67402521, 44, Finished, Available)

,max_LastUpdate
0,NaT


In [5]:
# parameters for this notebook
table_name  = "SalesLT.Customer"

In [6]:
# Connect to SQL Server
SQL_DBNAME="aworks"
SQL_DRIVER = "ODBC Driver 18 for SQL Server"

connectionString = f'DRIVER={SQL_DRIVER};SERVER={SQL_SERVER};DATABASE={SQL_DBNAME};UID={SQL_USER};PWD={SQL_PWD}'
conn = pyodbc.connect(connectionString)

cursor = conn.cursor()

In [7]:
import pandas as pd 
from datetime import datetime, timedelta

query = f"select MAX(ModifiedDate) as latest_modified_record from {table_name}"
sql_query = pd.read_sql_query (query, conn)

df = pd.DataFrame(sql_query, columns = ['latest_modified_record'])
for index, row in df.iterrows():
    print(row['latest_modified_record'])

2009-05-16 16:33:33.123000


C:\Users\dschlesinger\AppData\Local\Temp\ipykernel_32096\2960600840.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_query = pd.read_sql_query (query, conn)


In [54]:
# ingest the dataframe into the table
# maybe need to use the ingestion endpoint?

def ingest_to_kusto(dataframe):
    ingestTableCommand = f".ingest inline into table {table_name} with (ignoreFirstRecord=true) <| {dataframe.to_csv(index=False)} "
    response = kustoClient.execute(KUSTO_DATABASE, ingestTableCommand)
    print(dataframe_from_result_table(response.primary_results[0]))

StatementMeta(, 614ef43d-92fe-44b7-91e3-5d1c67402521, 58, Finished, Available)

In [17]:
# check if the copy is necessary
import pytz
utc=pytz.UTC

latest_modified_record = df['latest_modified_record'][0]
latest_modified_record = latest_modified_record.replace(tzinfo=utc)
# last_copy_date = last_copy_date.replace(tzinfo=utc)
last_copy_date = '2000-03-31 14:48:00'

last_copy_date = datetime.strptime(last_copy_date, '%Y-%m-%d %H:%M:%S')
last_copy_date = last_copy_date.replace(tzinfo=utc)

if latest_modified_record > last_copy_date:
    print("Data needs to be copied")
    copy_date = last_copy_date.strftime('%Y-%m-%d %H:%M:%S')
    query = f"select * from {table_name} where ModifiedDate > '{copy_date}'"
    print(query)
    sql_query = pd.read_sql_query (query, conn)

    sql_df = pd.DataFrame(sql_query)
    print(sql_df.head())
    # ingest_to_kusto(sql_df)
else:
    print("No need to copy data")

Data needs to be copied
select * from SalesLT.Customer where ModifiedDate > '2000-03-31 14:48:00'


C:\Users\dschlesinger\AppData\Local\Temp\ipykernel_32096\10536886.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_query = pd.read_sql_query (query, conn)


   CustomerID  NameStyle Title FirstName MiddleName    LastName Suffix  \
0           1      False   Mr.   Orlando         N.         Gee   None   
1           2      False   Mr.     Keith       None      Harris   None   
2           3      False   Ms.     Donna         F.    Carreras   None   
3           4      False   Ms.     Janet         M.       Gates   None   
4           5      False   Mr.      Lucy       None  Harrington   None   

                  CompanyName               SalesPerson  \
0                A Bike Store   adventure-works\pamela0   
1          Progressive Sports    adventure-works\david8   
2    Advanced Bike Components  adventure-works\jillian0   
3       Modular Cycle Systems  adventure-works\jillian0   
4  Metropolitan Sports Supply      adventure-works\shu0   

                   EmailAddress         Phone  \
0  orlando0@adventure-works.com  245-555-0173   
1    keith0@adventure-works.com  170-555-0127   
2    donna0@adventure-works.com  279-555-0130   
3   